In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Funções auxiliares

In [ ]:
import pandas as pd
import numpy as np
import random
import gc

# Definindo a semente
random.seed(123)

# Metadados referente ao conjunto de dados
def academy_generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

def remove_highly_correlated_features(df, threshold):
  # Calculate the correlation matrix
  corr_matrix = df.corr().abs()

  # Select the upper triangle of the correlation matrix
  upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

  # Identify columns to drop based on the threshold
  to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

  # Drop the columns
  df_reduced = df.drop(columns=to_drop)

  return df_reduced, to_drop

def amostragem(df,tamanho_amostra):
  amostra = df.sample(n=tamanho_amostra,random_state=42)
  return amostra

def vars_selection_previous_app(df,percentual_preenchimento,threshold,tamanho_amostragem,chave_principal):
  df_aux = df.drop(columns = chave_principal)
  amostra = amostragem(df_aux, tamanho_amostragem)
  metadata_df = pod_academy_generate_metadata(amostra)

  vars = metadata_df[metadata_df.percent_nulos <= percentual_preenchimento]['nome_variavel']
  df_reduced, dropped_features = remove_highly_correlated_features(amostra[vars], threshold=threshold)
  vars_selected = df_reduced.columns.to_list()
  vars_selected.append(chave_principal)
  df_selected = df[vars_selected]
  return df_selected

## Configuração do ambiente para utilização do Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# instalando a findspark
!pip install -q findspark

# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Minha Primeira Aplicação no Pyspark") \
        .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Leitura dos dados

In [ ]:
# Lê o arquivo Parquet
dados = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados/previous_application.csv",header=True)

# Mostra os dados
dados.show()

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-------------------+---------------------+------------------------+----------------------+--------------------+-------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+----------------+--------------------+-----------+----------------+--------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|  RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURPOSE

Nesse trecho, o código importa o módulo re, extrai os valores únicos da coluna NAME_CONTRACT_STATUS do DataFrame dados e aplica uma função que substitui espaços por sublinhados e converte os textos para letras maiúsculas, padronizando os nomes dos status antes de exibi-los em uma nova lista.

In [ ]:
import re

# Extrai os valores únicos da coluna 'NAME_CONTRACT_STATUS'
status = [row["NAME_CONTRACT_STATUS"] for row in dados.select("NAME_CONTRACT_STATUS").distinct().collect()]

# Função de tratamento
def tratamento_palavra(palavra):
    return re.sub(r'\s', '_', palavra).upper()

# Aplica a transformação a cada valor único
nova_lista = [tratamento_palavra(s) for s in status]

# Exibe a lista resultante
print(nova_lista)

['APPROVED', 'UNUSED_OFFER', 'CANCELED', 'REFUSED']


## Criação de flags para nos auxiliar na visão temporal dos dados

Nesse trecho, o código habilita o uso do SparkSQL criando uma temp view chamada “dados”, permitindo consultas SQL sobre o DataFrame. Em seguida, é criada uma nova view (df_temp_01) por meio de uma consulta que adiciona colunas indicando se cada registro pertence aos últimos 3, 6, 12, 24 ou 36 meses com base no campo DAYS_DECISION, facilitando análises temporais. Isso é feito para permitir segmentar os dados por períodos específicos e realizar comparações ou estudos de comportamento em diferentes janelas de tempo, o que é útil em análises de crédito ou de histórico de decisões.

In [ ]:
## Habilitando uso do SparkSQL
dados.createOrReplaceTempView("dados")

df_temp_01 = spark.sql("""
SELECT
    *,
      CASE
        WHEN DAYS_DECISION >= -90 THEN 1
        ELSE 0
    END AS ultimos_3_meses,
    CASE
        WHEN DAYS_DECISION >= -180 THEN 1
        ELSE 0
    END AS ultimos_6_meses,
    CASE
        WHEN DAYS_DECISION >= -360 THEN 1
        ELSE 0
    END AS ultimos_12_meses,
    CASE
        WHEN DAYS_DECISION >= -720 THEN 1
        ELSE 0
    END AS ultimos_24_meses,
    CASE
        WHEN DAYS_DECISION >= -1080 THEN 1
        ELSE 0
    END AS ultimos_36_meses
FROM dados
ORDER BY `SK_ID_PREV`;
""")
df_temp_01.createOrReplaceTempView("df_temp_01")
df_temp_01.show()

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+--------------------+---------------------+------------------------+----------------------+--------------------+-------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+----------------+----------------+--------------------+-----------+----------------+--------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+---------------+---------------+----------------+----------------+----------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|   RATE_D

## Join com bases (Pos Cash / Credit Card Balance / Installments Payments) contendo apenas as variáveis selecionadas

Nesse trecho, o código lê três arquivos CSV contendo dados agregados de diferentes fontes — pos_cash, credit_card_balance e installments — e os carrega como DataFrames Spark. Em seguida, realiza junções (joins) à esquerda com o DataFrame df_temp_01, usando a chave SK_ID_PREV como referência. Isso é feito para consolidar todas as informações relacionadas a cada contrato em um único DataFrame (df_temp_02), permitindo uma análise integrada e completa do histórico financeiro dos clientes.

In [ ]:
pos_cash = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/pos_cash_agg.csv",header=True)
credit_card_balance = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/credit_card_balance_agg.csv",header=True)
installments = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/installments_agg.csv",header=True)

In [ ]:
df_temp_02 = df_temp_01.join(pos_cash,"SK_ID_PREV",how='left').join(credit_card_balance,"SK_ID_PREV",how='left').join(installments,"SK_ID_PREV",how='left')
df_temp_02.show()

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-------------------+---------------------+------------------------+----------------------+--------------------+-------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+----------------+--------------------+-----------+----------------+--------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+---------------+---------------+----------------+----------------+----------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+-----------------------------------

## Sumarizar na visão cliente (Automatizada)

In [ ]:
colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

Nesse trecho, o código define as colunas de flag de janela temporal e monta uma lista de colunas para agregação (excluindo variáveis não desejadas), escolhe a primeira flag (ultimos_3_meses) e constrói expressões de agregação dinamicamente — para colunas que contenham DPD ou DAY calcula max e min condicionados à flag, e para as demais calcula sum, avg, max e min condicionados à flag, arredondando os resultados e gerando aliases padronizados; por fim agrupa por SK_ID_CURR aplicando essas agregações para produzir df_temp_03. Isso é feito para gerar features agregadas (totais, médias, extremos) por cliente referentes às operações dos últimos 3 meses (facilitando análise e modelagem de crédito), com nomes de coluna explicativos e valores arredondados prontos para uso em etapas seguintes.

In [ ]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when

# Definir as colunas para agregação
colunas_agregacao_total = df_temp_02.columns

variaveis_remover = ['SK_ID_CURR','SK_ID_PREV','DAYS_DECISION','NAME_CONTRACT_STATUS','WEEKDAY_APPR_PROCESS_START','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','DAYS_DECISION',
                     'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
                     'SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','NAME_YIELD_GROUP','PRODUCT_COMBINATION']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

expressoes_agregacao = []

flag = colunas_flags[0]

for coluna in colunas_agregacao_total:
  if 'DPD' in coluna or 'DAY' in coluna:
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_03 = df_temp_02.groupBy("SK_ID_CURR").agg(*expressoes_agregacao).orderBy("SK_ID_CURR")

# Mostrar o DataFrame resultante
df_temp_03.show()

Output hidden; open in https://colab.research.google.com to view.

Nesse trecho, o código reorganiza o DataFrame df_temp_03 em uma única partição usando repartition(1) e depois salva o resultado em um arquivo CSV no caminho especificado, sobrescrevendo qualquer arquivo existente (mode("overwrite")).

Isso é feito porque, por padrão, o Spark divide os dados em várias partições, o que resultaria em vários arquivos de saída. Ao usar repartition(1), garante-se que todos os dados fiquem em um único arquivo CSV consolidado, facilitando o armazenamento, o compartilhamento e o uso posterior do conjunto de dados em outras etapas de análise ou modelagem.

In [ ]:
df_temp_03 = df_temp_03.repartition(1)
df_temp_03.write.mode("overwrite").csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_1.csv",header=True)

Nesse trecho, o código gera agregações estatísticas por cliente considerando os últimos 6 meses, removendo colunas irrelevantes e calculando soma, média, máximo e mínimo (ou apenas máximo e mínimo para variáveis de dias e atrasos) apenas para registros dentro desse período. Em seguida, agrupa os resultados por cliente, ordena, consolida em uma única partição e salva o arquivo CSV final, criando indicadores resumidos úteis para análises e modelagem de crédito.

In [ ]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_02.columns

variaveis_remover = ['SK_ID_CURR','SK_ID_PREV','DAYS_DECISION','NAME_CONTRACT_STATUS','WEEKDAY_APPR_PROCESS_START','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','DAYS_DECISION',
                     'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
                     'SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','NAME_YIELD_GROUP','PRODUCT_COMBINATION']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

expressoes_agregacao = []

flag = colunas_flags[1]

for coluna in colunas_agregacao_total:
  if 'DPD' in coluna or 'DAY' in coluna:
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_04 = df_temp_02.groupBy("SK_ID_CURR").agg(*expressoes_agregacao).orderBy("SK_ID_CURR")

# Mostrar o DataFrame resultante
df_temp_04.show()

Output hidden; open in https://colab.research.google.com to view.

Nesse trecho, o código cria agregações estatísticas por cliente considerando os últimos 12 meses, removendo colunas irrelevantes e calculando soma, média, máximo e mínimo (ou apenas máximo e mínimo para variáveis de dias e atrasos) apenas para registros desse período. Depois, agrupa os resultados por cliente, ordena, consolida em uma única partição e salva o arquivo CSV final, gerando indicadores resumidos de comportamento e histórico de crédito no último ano.

In [ ]:
df_temp_04 = df_temp_04.repartition(1)
df_temp_04.write.mode("overwrite").csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_2.csv",header=True)

In [ ]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_02.columns

variaveis_remover = ['SK_ID_CURR','SK_ID_PREV','DAYS_DECISION','NAME_CONTRACT_STATUS','WEEKDAY_APPR_PROCESS_START','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','DAYS_DECISION',
                     'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
                     'SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','NAME_YIELD_GROUP','PRODUCT_COMBINATION']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

expressoes_agregacao = []

flag = colunas_flags[2]

for coluna in colunas_agregacao_total:
  if 'DPD' in coluna or 'DAY' in coluna:
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_05 = df_temp_02.groupBy("SK_ID_CURR").agg(*expressoes_agregacao).orderBy("SK_ID_CURR")

# Mostrar o DataFrame resultante
df_temp_05.show()

+----------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+------------------------------------

In [ ]:
df_temp_05 = df_temp_05.repartition(1)
df_temp_05.write.mode("overwrite").csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_3.csv",header=True)

Nesse trecho, o código gera agregações estatísticas por cliente considerando os últimos 24 meses, removendo colunas irrelevantes e calculando soma, média, máximo e mínimo (ou apenas máximo e mínimo para variáveis de dias e atrasos) somente para registros desse período. Em seguida, agrupa os resultados por cliente, ordena, consolida os dados em uma única partição e salva o arquivo CSV final, criando indicadores resumidos que refletem o comportamento de crédito dos últimos dois anos.

In [ ]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_02.columns

variaveis_remover = ['SK_ID_CURR','SK_ID_PREV','DAYS_DECISION','NAME_CONTRACT_STATUS','WEEKDAY_APPR_PROCESS_START','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','DAYS_DECISION',
                     'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
                     'SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','NAME_YIELD_GROUP','PRODUCT_COMBINATION']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

expressoes_agregacao = []

flag = colunas_flags[3]

for coluna in colunas_agregacao_total:
  if 'DPD' in coluna or 'DAY' in coluna:
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_06 = df_temp_02.groupBy("SK_ID_CURR").agg(*expressoes_agregacao).orderBy("SK_ID_CURR")

# Mostrar o DataFrame resultante
df_temp_06.show()

+----------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+------------------------------------

In [ ]:
df_temp_06 = df_temp_06.repartition(1)
df_temp_06.write.mode("overwrite").csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_4.csv",header=True)

Nesse trecho o código seleciona as colunas a serem agregadas (excluindo variáveis irrelevantes), monta expressões condicionais para calcular soma, média, máximo e mínimo (ou só máximo e mínimo para variáveis com DPD/DAY) considerando a flag ultimos_36_meses, agrupa por SK_ID_CURR para gerar métricas agregadas por cliente, exibe o resultado e finalmente consolida o DataFrame em uma única partição antes de salvar como CSV (previous_application_agg_5.csv) para uso posterior em análise ou modelagem.

In [ ]:
# Definir as colunas para agregação
colunas_agregacao_total = df_temp_02.columns

variaveis_remover = ['SK_ID_CURR','SK_ID_PREV','DAYS_DECISION','NAME_CONTRACT_STATUS','WEEKDAY_APPR_PROCESS_START','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','DAYS_DECISION',
                     'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
                     'SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','NAME_YIELD_GROUP','PRODUCT_COMBINATION']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

expressoes_agregacao = []

flag = colunas_flags[4]

for coluna in colunas_agregacao_total:
  if 'DPD' in coluna or 'DAY' in coluna:
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))
    expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APPLICATION"))

expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicar as expressões de agregação
df_temp_07 = df_temp_02.groupBy("SK_ID_CURR").agg(*expressoes_agregacao).orderBy("SK_ID_CURR")

# Mostrar o DataFrame resultante
df_temp_07.show()

+----------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+--------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+------------------------------------

In [ ]:
df_temp_07 = df_temp_07.repartition(1)
df_temp_07.write.mode("overwrite").csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_5.csv",header=True)

In [ ]:
# # Definir as colunas para agregação
colunas_agregacao_total = df_temp_02.columns

# variaveis_remover = ['SK_ID_CURR','SK_ID_PREV','DAYS_DECISION','NAME_CONTRACT_STATUS','WEEKDAY_APPR_PROCESS_START','NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','DAYS_DECISION',
#                      'NAME_PAYMENT_TYPE','CODE_REJECT_REASON','NAME_TYPE_SUITE','NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
#                      'SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','NAME_YIELD_GROUP','PRODUCT_COMBINATION']

# colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# colunas_flags = ['ultimos_3_meses','ultimos_6_meses','ultimos_12_meses','ultimos_24_meses','ultimos_36_meses']

# expressoes_agregacao = []

# for categoria in nova_lista:
#   for flag in colunas_flags:
#     for coluna in colunas_agregacao_total:
#       if 'DPD' in coluna or 'DAY' in coluna:
#         expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APPLICATION"))
#         expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"QT_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APPLICATION"))
#       else:
#         expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APPLICATION"))
#         expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APPLICATION"))
#         expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APPLICATION"))
#         expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f"VL_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APPLICATION"))

# expressoes_agregacao = tuple(expressoes_agregacao)

# # Aplicar as expressões de agregação
# df_temp_04 = df_temp_02.groupBy("SK_ID_PREV").agg(*expressoes_agregacao).orderBy("SK_ID_PREV")

# # Mostrar o DataFrame resultante
# df_temp_04.show()

Nesse trecho, o código lê as cinco bases agregadas (agg1 a agg5) que contêm indicadores calculados para diferentes janelas de tempo (3, 6, 12, 24 e 36 meses), usando spark.read.csv com as opções header=True e inferSchema=True para manter os nomes das colunas e identificar automaticamente os tipos de dados. Em seguida, as bases são unidas por meio de left joins usando a chave SK_ID_CURR, consolidando todas as informações em um único DataFrame (df_prev). Por fim, o resultado é salvo em formato CSV, sobrescrevendo versões anteriores, para formar uma base completa com todas as variáveis agregadas e prontas para análises ou modelagem.

In [ ]:
# Ler as bases agregadas
agg1 = spark.read.csv(
    "/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_1.csv",
    header=True,
    inferSchema=True
)

agg2 = spark.read.csv(
    "/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_2.csv",
    header=True,
    inferSchema=True
)

agg3 = spark.read.csv(
    "/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_3.csv",
    header=True,
    inferSchema=True
)

agg4 = spark.read.csv(
    "/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_4.csv",
    header=True,
    inferSchema=True
)

agg5 = spark.read.csv(
    "/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_agg_5.csv",
    header=True,
    inferSchema=True
)

# Fazer os joins pelo SK_ID_CURR
df_prev = (agg1
           .join(agg2, "SK_ID_CURR", "left")
           .join(agg3, "SK_ID_CURR", "left")
           .join(agg4, "SK_ID_CURR", "left")
           .join(agg5, "SK_ID_CURR", "left"))

# Salvar o consolidado no Drive
df_prev.write.mode("overwrite").csv(
    "/content/drive/MyDrive/Colab Notebooks/Projetos/Modelo_Credito_Application /Conjunto_de_Dados_Tratados/previous_application_completo_agg.csv",
    header=True
)
